In [1]:
import os
prefix = "/home/kreffert/"
prefix = "/pfs/data6/home/ma/ma_ma/ma_kreffert/" 
os.chdir(f'{prefix}Probabilistic_LTSF/BasicTS/')
from basicts.metrics import masked_mae, masked_mse, nll_loss, crps, Evaluator, quantile_loss, empirical_crps
from easytorch.device import set_device_type
from easytorch.utils import get_logger, set_visible_devices
# set the device type (CPU, GPU, or MLU)
device_type ='gpu'
gpus = '0'
set_device_type(device_type)
set_visible_devices(gpus)
from easydict import EasyDict
from tqdm import tqdm

# extract the paths to the configs and weights
import yaml
# /home/kreffert/Probabilistic_LTSF/notebooks/Final plots/weights.yaml
with open(f'{prefix}Probabilistic_LTSF/notebooks/Final plots/weights.yaml', 'r') as file:
    _configs = yaml.safe_load(file)
    
def reconstruct_paths(
    simplified_dict,
    _dataset=['ETTh1', 'ETTm1'],
    _models=['DLinear', 'PatchTST', 'DeepAR'],
    _dists=['q', 'iq', 'u', 'm'],
    _seeds=[0, 1, 2, 3, 4],
    _model_dist_map=None  # optional: {'DLinear': ['m'], 'DeepAR': ['u']}
):
    base_path = "final_weights/"
    dist_mapping = {"iq": "i_quantile", "u": "univariate", "m": "multivariate", "q": "quantile"}
    filtered_dict = {}

    for dataset, models in simplified_dict.items():
        if dataset not in _dataset:
            continue
        for model, dists in models.items():
            if _model_dist_map:
                if model not in _model_dist_map:
                    continue
            elif model not in _models:
                continue

            allowed_dists = _model_dist_map[model] if _model_dist_map else _dists

            for dist, seeds in dists.items():
                if dist not in allowed_dists:
                    continue
                _cfg = f"{dataset}_prob_quantile.py" if dist in ["q", "iq"] else f"{dataset}_prob.py"
                _ckpt = "_best_val_QL.pt" if dist in ["q", "iq"] else "_best_val_NLL.pt"
                mapped_dist = dist_mapping.get(dist, dist)

                for seed, path_suffix in seeds.items():
                    if seed not in _seeds or path_suffix is None:
                        continue
                    prefix = f"{base_path}{dataset}/{model}/{mapped_dist}/{seed}/{path_suffix}"
                    ckpt_path = f"{prefix}/{model}{_ckpt}"
                    if os.path.isfile(ckpt_path):
                        filtered_dict.setdefault(dataset, {}).setdefault(model, {}).setdefault(dist, {})[seed] = {
                            'cfg': f"{prefix}/{_cfg}",
                            'ckpt': f"{prefix}/{model}{_ckpt}"
                        }
                    # else:
                    #     print(f"{ckpt_path} not found.")

    return filtered_dict



_model_dist_map={'DLinear': ['m'], 'DeepAR': ['u']}
_models = ['DeepAR', 'DLinear'] #full_dict[dataset][model][dist][random_state]
_dataset = ['ETTh1']
_dists = ['u', 'm', 'q', 'iq']
_seeds = [2]
# _configs = reconstruct_paths(_configs, _dataset=_dataset, _models=_models, _dists=_dists, _seeds=_seeds, _model_dist_map=_model_dist_map)
_configs = reconstruct_paths(_configs)

2025-07-02 18:23:53.683016: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 18:23:53.696307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751473433.710994 1350581 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751473433.715652 1350581 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-02 18:23:53.732411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print(_configs)

{'ETTh1': {'DLinear': {'iq': {2: {'cfg': 'final_weights/ETTh1/DLinear/i_quantile/2/7e59031295d7db4c807d3b5f9db7bc80/ETTh1_prob_quantile.py', 'ckpt': 'final_weights/ETTh1/DLinear/i_quantile/2/7e59031295d7db4c807d3b5f9db7bc80/DLinear_best_val_QL.pt'}}, 'q': {2: {'cfg': 'final_weights/ETTh1/DLinear/quantile/2/a7f10e8c211a68a5aeb51fbaec3c28a5/ETTh1_prob_quantile.py', 'ckpt': 'final_weights/ETTh1/DLinear/quantile/2/a7f10e8c211a68a5aeb51fbaec3c28a5/DLinear_best_val_QL.pt'}}, 'm': {1: {'cfg': 'final_weights/ETTh1/DLinear/multivariate/1/aa5d43538e1d49f3c5367e942fdf55cb/ETTh1_prob.py', 'ckpt': 'final_weights/ETTh1/DLinear/multivariate/1/aa5d43538e1d49f3c5367e942fdf55cb/DLinear_best_val_NLL.pt'}, 2: {'cfg': 'final_weights/ETTh1/DLinear/multivariate/2/ab8ce283463444e38a4ab2c8b7fd3d83/ETTh1_prob.py', 'ckpt': 'final_weights/ETTh1/DLinear/multivariate/2/ab8ce283463444e38a4ab2c8b7fd3d83/DLinear_best_val_NLL.pt'}}}, 'DeepAR': {'iq': {2: {'cfg': 'final_weights/ETTh1/DeepAR/i_quantile/2/10b4108b626a9d27

In [3]:
import torch
import pickle
import gc
import torch  # assuming PyTorch is being used
from prob.prob_head import ProbabilisticHead 

def load_cfg(cfg, random_state=None):
    from easytorch.config import init_cfg
    # cfg path which start with dot will crash the easytorch, just remove dot
    while isinstance(cfg, str) and cfg.startswith(('./','.\\')):
        cfg = cfg[2:]
    # while ckpt_path.startswith(('./','.\\')):
    #     ckpt_path = ckpt_path[2:]
    
    # initialize the configuration
    cfg = init_cfg(cfg, save=False)
    return cfg
    
@torch.no_grad()
def _forward(runner, cfg, data, epoch: int = None, iter_num: int = None, train: bool = True, ims=False, **kwargs):
        """
        Performs the forward pass for training, validation, and testing. 

        Args:
            data (Dict): A dictionary containing 'target' (future data) and 'inputs' (history data) (normalized by self.scaler).
            epoch (int, optional): Current epoch number. Defaults to None.
            iter_num (int, optional): Current iteration number. Defaults to None.
            train (bool, optional): Indicates whether the forward pass is for training. Defaults to True.

        Returns:
            Dict: A dictionary containing the keys:
                  - 'inputs': Selected input features.
                  - 'prediction': Model predictions.
                  - 'target': Selected target features.

        Raises:
            AssertionError: If the shape of the model output does not match [B, L, N].
        """
        distribution_type = runner.distribution_type
        model_name = cfg["MODEL"]["NAME"]
        if distribution_type in ['gaussian', 'student_t', 'laplace', 'm_lr_gaussian']:
             prob_args = cfg['MODEL']['PARAM'].get('prob_args', None)
             prob_head = ProbabilisticHead(1, 1, distribution_type, prob_args=prob_args)
             sample = True
        elif distribution_type in ['i_quantile']:
            sample = True
        else:
            prob_head = None
            sample = False
        data = runner.preprocessing(data)

        # Preprocess input data
        future_data, history_data = data['target'], data['inputs']
        history_data = runner.to_running_device(history_data)  # Shape: [B, L, N, C]
        future_data = runner.to_running_device(future_data)    # Shape: [B, L, N, C]
        batch_size, length, num_nodes, _ = future_data.shape

        # Select input features
        history_data = runner.select_input_features(history_data)
        future_data_4_dec = runner.select_input_features(future_data)

        if not train:
            # For non-training phases, use only temporal features
            future_data_4_dec[..., 0] = torch.empty_like(future_data_4_dec[..., 0])

        # Forward pass through the model
        if not (model_name == 'DeepAR' and distribution_type in ['i_quantile']):
            model_return = runner.model(history_data=history_data, future_data=future_data_4_dec,
                                        batch_seen=iter_num, epoch=epoch, train=train)
        else:
            with torch.no_grad():
                model_return = runner.model.sample_trajectories(history_data=history_data, future_data=future_data_4_dec, num_samples=100)
                model_return = model_return.permute(1, 2, 3, 0)
        if distribution_type in ['gaussian', 'student_t', 'laplace', 'm_lr_gaussian']:
            if model_name == 'DeepAR':
                samples = runner.model.sample_trajectories(history_data=history_data, future_data=future_data_4_dec, num_samples=100)
            else:
                samples = prob_head.sample(model_return, num_samples=100, random_state=None) # shape torch.Size([num_samples, batch_size, output_len, num_series])
            samples = samples.permute(1, 2, 3, 0)
        elif distribution_type in ['i_quantile']:
            samples = model_return[..., -100:] # torch.Size([64, 720, 7, 100])
            model_return = model_return[..., :-100]

        # Parse model return
        if isinstance(model_return, torch.Tensor):
            model_return = {'prediction': model_return}
        if 'inputs' not in model_return:
            model_return['inputs'] = runner.select_target_features(history_data)
        if 'target' not in model_return:
            model_return['target'] = runner.select_target_features(future_data)
            
        # Ensure the output shape is correct
        assert list(model_return['prediction'].shape)[:3] == [batch_size, length, num_nodes], \
            f"The shape of the output is incorrect. Ensure it matches [B, L, N, C]. Current {list(model_return['prediction'].shape)[:3]} != {[batch_size, length, num_nodes]}"

        model_return = runner.postprocessing(model_return)
        if sample:
            model_return['samples'] = runner.scaler.inverse_transform(samples, head='quantile')
            assert list(model_return['samples'].shape)[:3] == [batch_size, length, num_nodes], \
            f"The shape of the output is incorrect. Ensure it matches [B, L, N, C]. Current {list(model_return['samples'].shape)} != {[batch_size, length, num_nodes]}"
        return model_return
            
@torch.no_grad()
def get_predictions(runner, cfg):
    print(cfg["MODEL"]["NAME"])
    # init test
    runner.test_interval = cfg['TEST'].get('INTERVAL', 1)
    runner.test_data_loader = runner.build_test_data_loader(cfg)

    runner.model.eval()
    prediction, target, inputs = [], [], []
    distribution_type = runner.distribution_type
    if distribution_type in ['gaussian', 'student_t', 'laplace', 'm_lr_gaussian']:
        sample = True
        samples = []
    if distribution_type in ['i_quantile']:
        sample = True
        samples = []
    else:
        sample = False
    for data in tqdm(runner.test_data_loader):
        # if model DeepAR -> forward with postprocessing of quantile and 100 sample trajectories!
        
        forward_return = _forward(runner, cfg, data, epoch=None, iter_num=None, train=False)
        if not runner.if_evaluate_on_gpu:
            forward_return['prediction'] = forward_return['prediction'].detach().cpu()
            forward_return['target'] = forward_return['target'].detach().cpu()
            forward_return['inputs'] = forward_return['inputs'].detach().cpu()
            if sample:
                forward_return['samples'] = forward_return['samples'].detach().cpu()
                

        prediction.append(forward_return['prediction'])
        target.append(forward_return['target'])
        inputs.append(forward_return['inputs'])
        if sample:
            samples.append(forward_return['samples'])

    prediction = torch.cat(prediction, dim=0)
    target = torch.cat(target, dim=0)
    inputs = torch.cat(inputs, dim=0)
    if sample:
        samples = torch.cat(samples, dim=0)
        return {'prediction': prediction, 'target': target, 'inputs': inputs, 'samples':samples}
    else:
        return {'prediction': prediction, 'target': target, 'inputs': inputs}
    # configs[dataset][model][dist][random_state]['returns_all'] = returns_all

def load_runner(configs):
    for dataset in configs.keys():
        for model in configs[dataset].keys():
            for dist in configs[dataset][model].keys():
                for random_state in configs[dataset][model][dist].keys():
                    # configs[dataset][model][dist][random_state]['cfg'] = load_cfg(configs[dataset][model][dist][random_state]['cfg'])
                    # cfg = configs[dataset][model][dist][random_state]['cfg']
                    cfg = load_cfg(configs[dataset][model][dist][random_state]['cfg'])
                    ckpt = configs[dataset][model][dist][random_state]['ckpt']
                    strict=True
                    runner = cfg['RUNNER'](cfg)
                    # setup the graph if needed
                    if runner.need_setup_graph:
                        runner.setup_graph(cfg=cfg, train=False)
                        
                    print(f'Loading model checkpoint from {ckpt}')
                    runner.load_model(ckpt_path=ckpt, strict=strict)
                    
                    # runner.test_pipeline(cfg=cfg, save_metrics=False, save_results=False)
                    # configs[dataset][model][dist][random_state]['runner'] = runner

                    # produce predictions
                    returns_all = get_predictions(runner, cfg)
                    print(returns_all['prediction'].shape)
                    print(returns_all['target'].shape)
                    print(returns_all['inputs'].shape)
                    print(returns_all['samples'].shape)
                    ckpt_dir = cfg['TRAIN'].get('CKPT_SAVE_DIR', 1)
                    print(f'saved under {ckpt_dir}/model_return.pkl')
                    with open(f'{ckpt_dir}/model_return.pkl', 'wb') as f:
                        pickle.dump(returns_all, f)
                    del runner
                    del cfg
                    del returns_all
                    gc.collect()
                    torch.cuda.empty_cache()
    return configs

# deepar = {'ETTh1': {'DeepAR': {'u': {2: {'cfg': 'final_weights/ETTh1/DeepAR/univariate/2/61d4606f2ce85d01757a4e462bf0517c/ETTh1_prob.py', 'ckpt': 'final_weights/ETTh1/DeepAR/univariate/2/61d4606f2ce85d01757a4e462bf0517c/DeepAR_best_val_NLL.pt'}}}}}
# deepar = {'ETTh1': {'DLinear': {'iq': {2: {'cfg': 'final_weights/ETTh1/DLinear/i_quantile/2/7e59031295d7db4c807d3b5f9db7bc80/ETTh1_prob_quantile.py', 'ckpt': 'final_weights/ETTh1/DLinear/i_quantile/2/7e59031295d7db4c807d3b5f9db7bc80/DLinear_best_val_QL.pt'}}}}}
deepar = {'ETTh1': {'DeepAR': {'iq': {2: {'cfg': 'final_weights/ETTh1/DeepAR/i_quantile/2/10b4108b626a9d27fad40eddcff3715a/ETTh1_prob_quantile.py', 'ckpt': 'final_weights/ETTh1/DeepAR/i_quantile/2/10b4108b626a9d27fad40eddcff3715a/DeepAR_best_val_QL.pt'}}}}}
deepar = {'ETTh1': {'DLinear': {'q': {2: {'cfg': 'final_weights/ETTh1/DLinear/quantile/2/a7f10e8c211a68a5aeb51fbaec3c28a5/ETTh1_prob_quantile.py', 'ckpt': 'final_weights/ETTh1/DLinear/quantile/2/a7f10e8c211a68a5aeb51fbaec3c28a5/DLinear_best_val_QL.pt'}}}}}
deepar = {'ETTh1': {'DeepAR': {'q': {2: {'cfg': 'final_weights/ETTh1/DeepAR/quantile/2/2ad27c696897fc216064f719a8bea178/ETTh1_prob_quantile.py', 'ckpt': 'final_weights/ETTh1/DeepAR/quantile/2/2ad27c696897fc216064f719a8bea178/DeepAR_best_val_QL.pt'}}}}}

_configs = load_runner(deepar) #_configs)

2025-07-02 18:24:16,799 - easytorch-env - INFO - Disable TF32 mode
2025-07-02 18:24:16,813 - easytorch-env - INFO - Use deterministic algorithms.
2025-07-02 18:24:16,814 - easytorch-env - INFO - Set cudnn deterministic.
2025-07-02 18:24:16,814 - easytorch - INFO - Set ckpt save dir: '/pfs/data6/home/ma/ma_ma/ma_kreffert/Probabilistic_LTSF/BasicTS/final_weightsETTh1/DeepAR/i_quantile/2/ae4525304ecb7ad6f13cf83b4f107bf5'
2025-07-02 18:24:16,815 - easytorch - INFO - Building model.
2025-07-02 18:24:17,254 - easytorch - INFO - Load model from : final_weights/ETTh1/DeepAR/i_quantile/2/10b4108b626a9d27fad40eddcff3715a/DeepAR_best_val_QL.pt
2025-07-02 18:24:17,255 - easytorch - INFO - Loading Checkpoint from 'final_weights/ETTh1/DeepAR/i_quantile/2/10b4108b626a9d27fad40eddcff3715a/DeepAR_best_val_QL.pt'
2025-07-02 18:24:17,278 - easytorch - INFO - Test dataset length: 2065


DeepAR
Loading model checkpoint from final_weights/ETTh1/DeepAR/i_quantile/2/10b4108b626a9d27fad40eddcff3715a/DeepAR_best_val_QL.pt
DeepAR


100%|██████████| 33/33 [20:12<00:00, 36.74s/it]


torch.Size([2065, 720, 7, 9])
torch.Size([2065, 720, 7, 1])
torch.Size([2065, 96, 7, 1])
torch.Size([2065, 720, 7, 100])
saved under /pfs/data6/home/ma/ma_ma/ma_kreffert/Probabilistic_LTSF/BasicTS/final_weightsETTh1/DeepAR/i_quantile/2/model_return.pkl


In [4]:
# evaluate the model_return pickle files
# evaluate the models
import torch
def vs_ensemble_torch(obs, fct, p=1.0):
    """
    Compute Variogram Score using PyTorch on GPU.
    obs: shape (..., D)
    fct: shape (..., M, D)
    """
    M = fct.shape[-2]

    # Compute ensemble variogram component
    fct_diff = fct.unsqueeze(-2) - fct.unsqueeze(-1)  # (B, M, D, D)
    # print(fct_diff.shape)
    vfct = (fct_diff.abs() ** p).sum(dim=-3) / M  # (B, D, D)
    # print(vfct.shape)
    # Compute observed variogram component
    obs_diff = obs.unsqueeze(-2) - obs.unsqueeze(-1)  # (B, D, D)
    vobs = (obs_diff.abs() ** p)  # (B, D, D)
    # print(vobs.shape)
    # print(vfct.shape)
    vs = ((vfct - vobs) ** 2).sum(dim=(-2, -1))  # (B,)
    return vs

def es_ensemble_torch(obs: torch.Tensor, fct: torch.Tensor) -> torch.Tensor:
    """
    Compute the energy score using PyTorch.
    
    Parameters:
    - obs: Tensor of shape (B, D)
    - fct: Tensor of shape (B, M, D)

    Returns:
    - Tensor of shape (B,) with energy scores
    """
    M = fct.shape[-2]

    # E_1: mean norm between forecast samples and the observation
    err_norm = torch.norm(fct - obs.unsqueeze(-2), dim=-1)  # (B, M)
    E_1 = err_norm.sum(dim=-1) / M  # (B,)

    # E_2: mean pairwise distance between forecast samples
    spread = fct.unsqueeze(-3) - fct.unsqueeze(-2)  # (B, M, M, D)
    spread_norm = torch.norm(spread, dim=-1)  # (B, M, M)
    E_2 = spread_norm.sum(dim=(-2, -1)) / (M**2) # (B,)

    return E_1 - 0.5 * E_2  # (B,)


def coverage(y, forecast):
    return np.mean(forecast >= y)
    
def sharpness(lower, upper, y):
    y_safe = np.where(np.abs(y) == 0, 1e-7, np.abs(y))  # Avoid division by zero
    return np.mean((upper - lower) / y_safe)

def quantile_score(y, forecast, alpha):
    """
    Quantile score for a given quantile level (alpha).
    Lower is better.
    """
    return np.mean(2 * ((y < forecast) - alpha) * (forecast - y))

def interval_score(y, lower, upper, alpha):
    """
    Interval score for central prediction intervals.
    Lower is better.
    """
    return np.mean((upper - lower) + 
                   (2 / alpha) * (lower - y) * (y < lower) + 
                   (2 / alpha) * (y - upper) * (y > upper))

def weighted_quantile_score(y, forecast_quantiles, quantile_levels):
    """
    Computes the weighted Quantile Score (wQS).
    y: array of shape (D,)
    forecast_quantiles: array of shape (len(quantile_levels), D)
    """
    D = y.shape[0]
    qs_sum = 0
    for i, alpha in enumerate(quantile_levels):
        qs = quantile_score(y, forecast_quantiles[i], alpha)
        qs_sum += qs
    y = np.where(np.abs(y) == 0, 1e-7, np.abs(y))  # Avoid division by zero
    wqs = qs_sum / np.mean(np.abs(y)) if np.sum(np.abs(y)) != 0 else np.nan
    return wqs / len(quantile_levels)


def weighted_interval_score(y, forecast_quantiles, interval_bounds):
    """
    Computes the Weighted Interval Score (WIS).
    y: array of shape (D,)
    forecast_quantiles: array of shape (len(quantile_levels), D)
    interval_bounds: dict like {0.5: (0.25, 0.75), ...}
    """
    median = forecast_quantiles[quantile_levels.index(0.5)]
    wis = 0.5 * np.mean(np.abs(y - median))  # w0 * |y - median|

    for alpha in interval_bounds:
        lower_idx = quantile_levels.index(interval_bounds[alpha][0])
        upper_idx = quantile_levels.index(interval_bounds[alpha][1])
        lower = forecast_quantiles[lower_idx]
        upper = forecast_quantiles[upper_idx]
        iscore = interval_score(y, lower, upper, alpha=1 - alpha)
        wis += (1 - alpha) * iscore / 2  # wk = alpha/2 = (1 - alpha)/2

    K = len(interval_bounds)
    return wis / (K + 0.5)

In [5]:
import scoringrules as sr
from tqdm import tqdm
import numpy as np
paths = ['/pfs/data6/home/ma/ma_ma/ma_kreffert/Probabilistic_LTSF/BasicTS/final_weightsETTh1/DeepAR/univariate/2/model_return.pkl',
        '/pfs/data6/home/ma/ma_ma/ma_kreffert/Probabilistic_LTSF/BasicTS/final_weightsETTh1/DLinear/multivariate/2/model_return.pkl']

    
device = 'cuda:0'
quantile_levels = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]
interval_bounds = {0.5: (0.25, 0.75), 0.67: (0.165, 0.835), 0.95: (0.025, 0.975), 0.99: (0.005, 0.995)}

metrics = ['crps', 'crps_sum', 'crps_sum_q', 'crps_q', 'vs_05', 'vs_1', 'vs_2', 'es', 'wqs', 'wis']
metrics = metrics + [f"{m}{quantile}" for quantile in quantile_levels for m in ['coverage_', 'qs_']]
metrics = metrics + [f"{m}{interval}" for interval in interval_bounds.keys() for m in ['sharpness_', 'is_']]


results = {}
for metric in metrics:
    results[metric] = [] 

path=paths[0]
with open(path, 'rb') as f:
    model_return = pickle.load(f)
# inputs = model_return['inputs'].to(device)
target = model_return['target'].to(device)
samples = model_return['samples'].to(device)
# Create DataLoader
from torch.utils.data import TensorDataset, DataLoader
batch_size = 64
dataset = TensorDataset(samples, target)
data_iter = DataLoader(dataset, batch_size=batch_size, shuffle=False)
for iter_index, (samples_batch, target_batch) in tqdm(enumerate(data_iter), total=len(data_iter)):
    samples_batch = samples_batch.to(device)#.permute(0, 2, 3, 1)
    target_batch = target_batch.to(device).squeeze(-1)
    results['crps'].append(np.mean(sr.crps_ensemble(target_batch.detach().cpu(), samples_batch.detach().cpu(), estimator='pwm')))
    results['crps_sum'] = np.mean(sr.crps_ensemble(target_batch.detach().cpu().sum(axis=-1), samples_batch.detach().cpu().sum(axis=-2), estimator='pwm'))
    
    # prediction=s
    samples_batch = samples_batch.permute(0, 1, 3, 2)
    results['vs_05'].append(torch.mean(vs_ensemble_torch(target_batch.to(device), samples_batch.to(device), p=0.5)).cpu().numpy())
    # vs_1 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=1.0, backend='numba'))
    results['vs_1'].append(torch.mean(vs_ensemble_torch(target_batch.to(device), samples_batch.to(device), p=1)).cpu().numpy())
    results['vs_2'].append(torch.mean(vs_ensemble_torch(target_batch.to(device), samples_batch.to(device), p=2)).cpu().numpy())
    results['es'].append(torch.mean(es_ensemble_torch(target_batch.to(device), samples_batch.to(device))).cpu().numpy())
    
    # # now determine the quantiles of the forecasts and determine the quantile metrics
    samples_batch = samples_batch.permute(0, 1, 3, 2).cpu().numpy()
    target_batch = target_batch.cpu().numpy()
    
    quantiles = np.quantile(samples_batch.sum(axis=-2), quantile_levels, axis=-1)
    results['crps_sum_q'].append(np.mean(sr.crps_quantile(target_batch.sum(axis=-1), np.transpose(quantiles, (1, 2, 0)), quantile_levels)))
    
    quantiles = np.quantile(samples_batch, quantile_levels, axis=-1)
    results['crps_q'].append(np.mean(sr.crps_quantile(target_batch, np.transpose(quantiles, (1, 2, 3, 0)), quantile_levels)))
    
    # # evaluate interval metrics
    for interval in interval_bounds.keys():
        lower = torch.tensor(quantiles[quantile_levels.index(interval_bounds[interval][0])]).numpy()
        upper = torch.tensor(quantiles[quantile_levels.index(interval_bounds[interval][1])]).numpy()
        results[f'sharpness_{interval}'].append(sharpness(lower, upper, target_batch))
        results[f'is_{interval}'].append(interval_score(target_batch, lower, upper, alpha=1 - interval))

    for quantile in quantile_levels:
        forecast = quantiles[quantile_levels.index(quantile)]
        results[f'coverage_{quantile}'].append(coverage(target_batch, forecast))
        results[f'qs_{quantile}'].append(quantile_score(target_batch, forecast, alpha=quantile))

    # # Compute WQS and WIS
    results['wqs'].append(weighted_quantile_score(target_batch, quantiles, quantile_levels))
    results['wis'].append(weighted_interval_score(target_batch, quantiles, interval_bounds))
    
for metric in results.keys():
    results[metric] = round(np.mean(results[metric]), 4)
print(results)

100%|██████████| 33/33 [01:02<00:00,  1.90s/it]

{'crps': np.float64(3.7991), 'crps_sum': np.float64(8.1743), 'crps_sum_q': np.float64(5.4013), 'crps_q': np.float64(2.5017), 'vs_05': np.float32(92.5215), 'vs_1': np.float32(3371.6697), 'vs_2': np.float32(2696077.2), 'es': np.float32(15.5537), 'wqs': np.float64(0.5429), 'wis': np.float64(2.5017), 'coverage_0.005': np.float64(0.0127), 'qs_0.005': np.float64(0.1565), 'coverage_0.025': np.float64(0.0584), 'qs_0.025': np.float64(0.7019), 'coverage_0.165': np.float64(0.169), 'qs_0.165': np.float64(3.8028), 'coverage_0.25': np.float64(0.1906), 'qs_0.25': np.float64(4.721), 'coverage_0.5': np.float64(0.4002), 'qs_0.5': np.float64(5.4637), 'coverage_0.75': np.float64(0.6762), 'qs_0.75': np.float64(4.0574), 'coverage_0.835': np.float64(0.7659), 'qs_0.835': np.float64(2.9832), 'coverage_0.975': np.float64(0.9738), 'qs_0.975': np.float64(0.5076), 'coverage_0.995': np.float64(0.996), 'qs_0.995': np.float64(0.1212), 'sharpness_0.5': np.float64(645551.196), 'is_0.5': np.float64(17.5568), 'sharpness_

In [6]:
import scoringrules as sr
ground_truth = torch.tensor(samples[:, 51:])

device = 'cuda:0'
quantile_levels = [0.005, 0.025, 0.165, 0.25, 0.5, 0.75, 0.835, 0.975, 0.995]
interval_bounds = {0.5: (0.25, 0.75), 0.67: (0.165, 0.835), 0.95: (0.025, 0.975), 0.99: (0.005, 0.995)}
metrics = ['crps', 'vs_05', 'vs_1', 'vs_2', 'es', 'wqs', 'wis']
metrics = metrics + [f"{m}{quantile}" for quantile in quantile_levels for m in ['coverage_', 'qs_']]
metrics = metrics + [f"{m}{interval}" for interval in interval_bounds.keys() for m in ['sharpness_', 'is_']]
sample_list = [dms_samples, m_dms_samples, m_dms_2_samples, ims_samples]
results = {'DMS': {}, 'M_DMS':{}, 'M_DMS_2':{}, 'IMS':{}}
for i, s in enumerate(sample_list):
    key = list(results.keys())[i]
    for metric in metrics:
        results[key][metric] = [] 
    for g in range(ground_truth.shape[0]):
        target = ground_truth[g, :]
        prediction = s.permute(1, 0)
        results[key]['crps'].append(np.mean(sr.crps_ensemble(target.detach().cpu(), prediction.detach().cpu(), estimator='pwm')))
        prediction=s
        results[key]['vs_05'].append(torch.mean(vs_ensemble_torch(target.to(device), prediction.to(device), p=0.5)).cpu().numpy())
        # vs_1 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=1.0, backend='numba'))
        results[key]['vs_1'].append(torch.mean(vs_ensemble_torch(target.to(device), prediction.to(device), p=1)).cpu().numpy())
        results[key]['vs_2'].append(torch.mean(vs_ensemble_torch(target.to(device), prediction.to(device), p=2)).cpu().numpy())
        results[key]['es'].append(torch.mean(es_ensemble_torch(target.to(device), prediction.to(device))).cpu().numpy())
        
        # now determine the quantiles of the forecasts and determine the quantile metrics
        quantiles = np.quantile(prediction, quantile_levels, axis=0)
        
        # evaluate interval metrics
        for interval in interval_bounds.keys():
            lower = torch.tensor(quantiles[quantile_levels.index(interval_bounds[interval][0])]).numpy()
            upper = torch.tensor(quantiles[quantile_levels.index(interval_bounds[interval][1])]).numpy()
            results[key][f'sharpness_{interval}'].append(sharpness(lower, upper, target.numpy()))
            results[key][f'is_{interval}'].append(interval_score(target.numpy(), lower, upper, alpha=1 - interval))

        for quantile in quantile_levels:
            forecast = quantiles[quantile_levels.index(quantile)]
            results[key][f'coverage_{quantile}'].append(coverage(target.numpy(), forecast))
            results[key][f'qs_{quantile}'].append(quantile_score(target.numpy(), forecast, alpha=quantile))

        # Compute WQS and WIS
        results[key]['wqs'].append(weighted_quantile_score(target.numpy(), quantiles, quantile_levels))
        results[key]['wis'].append(weighted_interval_score(target.numpy(), quantiles, interval_bounds))

        
    for metric in results[key].keys():
        results[key][metric] = round(np.mean(results[key][metric]), 4)
print(results)

/scratch/slurm_tmpdir/job_629247/ipykernel_1350581/1202630747.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ground_truth = torch.tensor(samples[:, 51:])


NameError: name 'dms_samples' is not defined

In [ ]:
def get_samples(X_train, dms_model, m_dms_model, m_dms_2_model, ims_model, num_samples=50, random_state=42):
    # dms samples
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    prefix= X_train[:1, :].to(device)
    with torch.no_grad():
        dms_model.eval()
        head_output = dms_model(prefix)
        dms_samples = dms_model.sample(head_output, num_samples=num_samples, random_state=random_state)
        dms_samples = dms_samples.squeeze()

    with torch.no_grad():
        m_dms_model.eval()
        head_output = m_dms_model(prefix)
        m_dms_samples = m_dms_model.sample(head_output, num_samples=num_samples, random_state=random_state)
        m_dms_samples = m_dms_samples.squeeze()

    with torch.no_grad():
        m_dms_2_model.eval()
        head_output = m_dms_2_model(prefix)
        m_dms_2_samples = m_dms_2_model.sample(head_output, num_samples=num_samples, random_state=random_state)
        m_dms_2_samples = m_dms_2_samples.squeeze()
    
    
    with torch.no_grad():
        model_ims.eval()
        prefix_tensor = torch.tensor(prefix, dtype=torch.float32).unsqueeze(-1).to(device)
        ims_samples, sigmas_ims = model_ims.forecast(prefix_tensor, steps=51, use_mean=False, num_samples=num_samples)
        ims_samples = ims_samples.squeeze()

    return dms_samples, m_dms_samples, m_dms_2_samples, ims_samples
   
    


def plot_samples(X_train, Y_train, dms_samples, m_dms_samples, m_dms_2_samples, ims_samples):
    custom_text_color = "#333333"
    custom_bg = "#FDFDFF"
    
    custom_theme = {
        'axes.facecolor': custom_bg,
        'axes.edgecolor': 'black',
        'axes.grid': False,
        'grid.color': '#dcdcdc',
        'grid.linestyle': '-',
        'xtick.color': custom_text_color,
        'ytick.color': custom_text_color,
        'axes.labelcolor': custom_text_color,
        'text.color': custom_text_color
    }
    sns.set_theme(style='white')
    sns.set_style("white", rc=custom_theme)
    fig, axs = plt.subplots(nrows=5, figsize=(20, 20), sharey=True, sharex=True)

    # ground truth trajectory
    # lambda x: np.sin(50/4 * (x + self.prefix_len))     # World 0: continues sine
    # lambda x: -np.sin(50/4 * (x + self.prefix_len))    # World 2: cosine
    prefix = X_train[:1, :]
    offset = 5
    world_1 = [np.sin(50/4 * (x + 51)) for x in np.arange(prefix.shape[1]-offset, prefix.shape[1] + Y_train.shape[1]-offset)]
    world_2 = [-np.sin(50/4 * (x + 51)) for x in np.arange(prefix.shape[1]-offset, prefix.shape[1] + Y_train.shape[1]-offset)]

    
    # ground truth samples
    for s in range(Y_train.shape[0]):
        if s ==0:
            axs[0].plot(np.arange(prefix.shape[1], prefix.shape[1]+ Y_train.shape[1]), Y_train[s, :], color='black', alpha=0.2, label='Ground Truth Samples')
        else:
            axs[0].plot(np.arange(prefix.shape[1], prefix.shape[1]+ Y_train.shape[1]), Y_train[s, :], color='black', alpha=0.2)
    axs[0].plot(np.arange(prefix.shape[1], prefix.shape[1]+ Y_train.shape[1]), world_1, color='orange', linestyle='--', linewidth=2.5, label="$\sin(x)$-Base")
    axs[0].plot(np.arange(prefix.shape[1], prefix.shape[1]+ Y_train.shape[1]), world_2, color='orange', linestyle='--', linewidth=2.5, label="$-1\cdot\sin(x)$-Base")

    # dms samples
    # axs[1].plot(np.arange(prefix.shape[1]), prefix[0, :])
    for s in range(dms_samples.shape[0]):
        if s ==0:
            axs[1].plot(np.arange(prefix.shape[1], prefix.shape[1]+ dms_samples.shape[1]), dms_samples[s, :], color='blue', alpha=0.2, label='Sample Predictions')
        else:
            axs[1].plot(np.arange(prefix.shape[1], prefix.shape[1]+ dms_samples.shape[1]), dms_samples[s, :], color='blue', alpha=0.2)
    axs[1].plot(np.arange(prefix.shape[1], prefix.shape[1]+ dms_samples.shape[1]), world_1, color='orange', linestyle='--', linewidth=2.5)
    axs[1].plot(np.arange(prefix.shape[1], prefix.shape[1]+ dms_samples.shape[1]), world_2, color='orange', linestyle='--', linewidth=2.5)
    
        
    # dms multivariate samples
    # axs[2].plot(np.arange(prefix.shape[1]), prefix[0, :])
    for s in range(m_dms_samples.shape[0]):
        axs[2].plot(np.arange(prefix.shape[1], prefix.shape[1]+ m_dms_samples.shape[1]), m_dms_samples[s, :], color='blue', alpha=0.2)
    axs[2].plot(np.arange(prefix.shape[1], prefix.shape[1]+ m_dms_samples.shape[1]), world_1, color='orange', linestyle='--', linewidth=2.5)
    axs[2].plot(np.arange(prefix.shape[1], prefix.shape[1]+ m_dms_samples.shape[1]), world_2, color='orange', linestyle='--', linewidth=2.5)
    # axs[2].
    for s in range(m_dms_2_samples.shape[0]):
        axs[3].plot(np.arange(prefix.shape[1], prefix.shape[1]+ m_dms_2_samples.shape[1]), m_dms_2_samples[s, :], color='blue', alpha=0.2)
    axs[3].plot(np.arange(prefix.shape[1], prefix.shape[1]+ m_dms_2_samples.shape[1]), world_1, color='orange', linestyle='--', linewidth=2.5)
    axs[3].plot(np.arange(prefix.shape[1], prefix.shape[1]+ m_dms_2_samples.shape[1]), world_2, color='orange', linestyle='--', linewidth=2.5)
    # axs[2].
    
    # ims samples
    for s in range(ims_samples.shape[0]):
        axs[4].plot(np.arange(prefix.shape[1], prefix.shape[1]+ ims_samples.shape[1]), ims_samples[s, :], color='blue', alpha=0.2)
    axs[4].plot(np.arange(prefix.shape[1], prefix.shape[1]+ ims_samples.shape[1]), world_1, color='orange', linestyle='--', linewidth=2.5)
    axs[4].plot(np.arange(prefix.shape[1], prefix.shape[1]+ ims_samples.shape[1]), world_2, color='orange', linestyle='--', linewidth=2.5)
    
    for ax in axs:
        ax.axvline(prefix.shape[1], color="black", linestyle="--", label="Branching Point")
        ax.set_ylim(-2, 2) 
        ax.tick_params(axis='both', labelsize=14)
        # ax.set_ylabel('Value', fontsize=20, fontweight='bold')
    axs[-1].set_xlabel('Time', fontsize=20, fontweight='bold')
    labels = ['Ground Truth', 'DMS Model (univariate)', 'DMS Model (multivariate-full rank)', 'DMS Model (multivariate-low rank)', 'IMS Model']
    for i, ax in enumerate(axs):
        ax.text(
            0.5, 0.97, labels[i],
            transform=ax.transAxes,
            fontsize=18,
            fontweight='bold',
            verticalalignment='top',
            horizontalalignment = 'center',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='black')
        )
    plt.tight_layout()
    sns.despine(left=False, bottom=False)
    # Create a single legend for the entire figure
    # Collect handles and labels from the last subplot (they should be consistent across all subplots)
    # Collect all handles and labels from all subplots
    all_handles = []
    all_labels = []
    for ax in axs:
        handles, labels = ax.get_legend_handles_labels()
        all_handles.extend(handles)
        all_labels.extend(labels)
    
    # Remove duplicates while preserving order
    from collections import OrderedDict
    from matplotlib.lines import Line2D
    unique = list(OrderedDict.fromkeys(zip(all_handles, all_labels)))
    handles, labels = zip(*unique)
    
    # Create proxy handles with alpha=1 for the legend
    legend_handles = []
    for h in handles[:5]:
        if isinstance(h, Line2D):
            legend_handles.append(Line2D([], [], color=h.get_color(), linestyle=h.get_linestyle(), linewidth=h.get_linewidth(), alpha=1))
        else:
            legend_handles.append(h)
    # handles_1, labels_1 = axs[1].get_legend_handles_labels()
    # handles = [handles[:], handles_1[:]]
    # handles = [handles[i] for i in [0, 1, 4, 7]]
    # labels = [labels[i] for i in [0]]+ ['Cooling', 'Idle', 'Heavy Load']
    # Place the legend outside the subplots
    fig.legend(legend_handles, labels[:5], loc='lower center', ncol=1, #len(labels), 
              bbox_to_anchor=(0.9, 0.77), fontsize=14, frameon=True, framealpha=1)

X_train, Y_train = build_dms_dataset(samples, 50, 51)
dms_samples, m_dms_samples, m_dms_2_samples, ims_samples = get_samples(X_train, g_dms, m_lr_dms, m_dms_2_model, model_ims, num_samples=20, random_state=42)
X_train, Y_train, dms_samples, m_dms_samples, m_dms_2_samples, ims_samples = X_train.cpu(), Y_train.cpu(), dms_samples.cpu(), m_dms_samples.cpu(), m_dms_2_samples.cpu(), ims_samples.cpu()
plot_samples(X_train, Y_train, dms_samples, m_dms_samples, m_dms_2_samples, ims_samples)
plt.savefig("Synthetic_plots.pdf", format='pdf', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.ndimage import label
import numpy as np

from sklearn.neighbors import KernelDensity

def get_kde_hdr_intervals(x, level=0.5, bandwidth=0.1, grid_size=1000):
    """
    Estimate HDR intervals using Kernel Density Estimation.
    """
    x = x.flatten()
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(x[:, None])
    
    x_grid = np.linspace(np.min(x) - 0.5, np.max(x) + 0.5, grid_size)
    log_density = kde.score_samples(x_grid[:, None])
    density = np.exp(log_density)
    
    idx_sorted = np.argsort(-density)
    cumulative = np.cumsum(density[idx_sorted])
    cumulative /= cumulative[-1]

    threshold_idx = idx_sorted[cumulative <= level]
    hdr_mask = np.zeros_like(density, dtype=bool)
    hdr_mask[threshold_idx] = True
    
    labeled, num_features = label(hdr_mask)
    intervals = []
    for i in range(1, num_features + 1):
        region_idx = np.where(labeled == i)[0]
        if len(region_idx) > 0:
            low = x_grid[region_idx[0]]
            high = x_grid[region_idx[-1]]
            intervals.append((low, high))
    return intervals

def plot_sample_intervals(X_train, Y_train, dms_samples, m_dms_samples, m_dms_2_samples, ims_samples):
    custom_text_color = "#333333"
    custom_bg = "#FDFDFF"
    custom_theme = {
        'axes.facecolor': custom_bg,
        'axes.edgecolor': 'black',
        'axes.grid': False,
        'grid.color': '#dcdcdc',
        'xtick.color': custom_text_color,
        'ytick.color': custom_text_color,
        'axes.labelcolor': custom_text_color,
        'text.color': custom_text_color
    }

    sns.set_theme(style='white')
    sns.set_style("white", rc=custom_theme)
    fig, axs = plt.subplots(nrows=5, figsize=(20, 20), sharey=True, sharex=True)

    prefix = X_train[:1, :]
    offset = 5
    t = np.arange(prefix.shape[1], prefix.shape[1] + Y_train.shape[1])
    world_1 = [np.sin(50/4 * (x + 51)) for x in np.arange(prefix.shape[1]-offset, prefix.shape[1] + Y_train.shape[1]-offset)]
    world_2 = [-np.sin(50/4 * (x + 51)) for x in np.arange(prefix.shape[1]-offset, prefix.shape[1] + Y_train.shape[1]-offset)]

    def plot_ci(ax, samples, color, label=None):
        median = np.median(samples, axis=0)
        p25 = np.percentile(samples, 25, axis=0)
        p75 = np.percentile(samples, 75, axis=0)
        p5 = np.percentile(samples, 5, axis=0)
        p95 = np.percentile(samples, 95, axis=0)

        ax.plot(t, median, color=color, label=label, linewidth=2)
        ax.fill_between(t, p25, p75, color=color, alpha=0.3, label="50% CI" if label else None)
        ax.fill_between(t, p5, p95, color=color, alpha=0.1, label="90% CI" if label else None)
    
    def get_hdr_intervals(x, density, level=0.5):
        n_samples = len(x)
        n_hdr = int(n_samples * level)
        top_idx = np.argsort(density)[-n_hdr:]
        hdr_points = np.zeros(n_samples, dtype=bool)
        hdr_points[top_idx] = True
    
        # Sort by x for connected component analysis
        sorted_idx = np.argsort(x.flatten())
        mask_sorted = hdr_points[sorted_idx]
        
        # Label contiguous high-density regions
        labeled, num_features = label(mask_sorted)
        intervals = []
        for i in range(1, num_features + 1):
            region_idx = sorted_idx[labeled == i]
            region_vals = x[region_idx]
            intervals.append((np.min(region_vals), np.max(region_vals)))
        return intervals
    
    def plot_ci_knn(ax, samples, color, label=None, alpha_50=0.3, alpha_90=0.1, k=10):
        n_samples, n_timesteps = samples.shape
        # t = np.arange(n_timesteps)
        median = np.median(samples, axis=0)
    
        for t_idx in range(n_timesteps):
            x = samples[:, t_idx].reshape(-1, 1)
            nbrs = NearestNeighbors(n_neighbors=k).fit(x)
            distances, _ = nbrs.kneighbors(x)
            density = -np.mean(distances, axis=1)  # Higher density = less distance
    
            # Plot 50% HDR
            intervals_50 = get_hdr_intervals(x, density, level=0.5)
            for low, high in intervals_50:
                ax.fill_between([t[t_idx]-0.4, t[t_idx]+0.4], [low, low], [high, high],
                                color=color, alpha=alpha_50)
    
            # Plot 90% HDR
            intervals_90 = get_hdr_intervals(x, density, level=0.9)
            for low, high in intervals_90:
                ax.fill_between([t[t_idx]-0.4, t[t_idx]+0.4], [low, low], [high, high],
                                color=color, alpha=alpha_90)
    
        ax.plot(t, median, color=color, label=label, linewidth=2)
    def plot_ci_kde(ax, samples, color, label=None, alpha_50=0.3, alpha_90=0.1, bandwidth=0.1):
        n_samples, n_timesteps = samples.shape
        median = np.median(samples, axis=0)
        
        for t_idx in range(n_timesteps):
            x = samples[:, t_idx]
            
            # Plot 50% HDR
            intervals_50 = get_kde_hdr_intervals(x, level=0.5, bandwidth=bandwidth)
            for low, high in intervals_50:
                ax.fill_between([t[t_idx]-0.4, t[t_idx]+0.4], [low, low], [high, high],
                                color=color, alpha=alpha_50)
            
            # Plot 90% HDR
            intervals_90 = get_kde_hdr_intervals(x, level=0.9, bandwidth=bandwidth)
            for low, high in intervals_90:
                ax.fill_between([t[t_idx]-0.4, t[t_idx]+0.4], [low, low], [high, high],
                                color=color, alpha=alpha_90)
        
        ax.plot(t, median, color=color, label=label, linewidth=2)



    # Ground Truth
    plot_ci_kde(axs[0], Y_train.numpy(), 'black', label='Ground Truth Median')
    # plot_ci(axs[0], Y_train.numpy(), 'black', label='Ground Truth Median')
    
    axs[0].plot(t, world_1, color='orange', linestyle='--', linewidth=2.5, label="$\sin(x)$-Base")
    axs[0].plot(t, world_2, color='orange', linestyle='--', linewidth=2.5, label="$-1\cdot\sin(x)$-Base")

    # DMS Models
    plot_ci_kde(axs[1], dms_samples.numpy(), 'blue')
    plot_ci_kde(axs[2], m_dms_samples.numpy(), 'blue')
    plot_ci_kde(axs[3], m_dms_2_samples.numpy(), 'blue')
    plot_ci_kde(axs[4], ims_samples.numpy(), 'blue')
    # plot_ci(axs[1], dms_samples.numpy(), 'blue')
    # plot_ci(axs[2], m_dms_samples.numpy(), 'blue')
    # plot_ci(axs[3], m_dms_2_samples.numpy(), 'blue')
    # plot_ci(axs[4], ims_samples.numpy(), 'blue')

    
    for i, ax in enumerate(axs):
        ax.plot(t, world_1, color='orange', linestyle='--', linewidth=2.5)
        ax.plot(t, world_2, color='orange', linestyle='--', linewidth=2.5) 
        ax.axvline(prefix.shape[1], color="black", linestyle="--", label="Branching Point")
        ax.set_ylim(-2, 2)
        ax.tick_params(axis='both', labelsize=14)

    axs[-1].set_xlabel('Time', fontsize=20, fontweight='bold')
    labels = ['Ground Truth', 'DMS Model (univariate)', 'DMS Model (multivariate-full rank)', 'DMS Model (multivariate-low rank)', 'IMS Model']
    for i, ax in enumerate(axs):
        ax.text(
            0.5, 0.97, labels[i],
            transform=ax.transAxes,
            fontsize=18,
            fontweight='bold',
            verticalalignment='top',
            horizontalalignment = 'center',
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='black')
        )

    plt.tight_layout()
    sns.despine(left=False, bottom=False)

    # Legend (only first subplot)
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.85, 0.79), fontsize=14, frameon=True, framealpha=1)
dms_samples, m_dms_samples, m_dms_2_samples, ims_samples = get_samples(X_train, g_dms, m_lr_dms, m_dms_2_model, model_ims, num_samples=100, random_state=42)
X_train, Y_train, dms_samples, m_dms_samples, m_dms_2_samples, ims_samples = X_train.cpu(), Y_train.cpu(), dms_samples.cpu(), m_dms_samples.cpu(), m_dms_2_samples.cpu(), ims_samples.cpu()
plot_sample_intervals(X_train, Y_train, dms_samples, m_dms_samples, m_dms_2_samples, ims_samples)
plt.savefig("Synthetic_plots_intervals_knn.pdf", format='pdf', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import torch
@torch.no_grad()
def get_predictions(configs):
    for dataset in configs.keys():
        for model in configs[dataset].keys():
            for dist in configs[dataset][model].keys():
                for random_state in configs[dataset][model][dist].keys():
                    runner = configs[dataset][model][dist][random_state]['runner']
                    cfg = configs[dataset][model][dist][random_state]['cfg']
                    # init test
                    runner.test_interval = cfg['TEST'].get('INTERVAL', 1)
                    runner.test_data_loader = runner.build_test_data_loader(cfg)
                
                    runner.model.eval()
                    prediction, target, inputs = [], [], []
                
                    for data in tqdm(runner.test_data_loader):
                        # if model DeepAR -> forward with postprocessing of quantile and 100 sample trajectories!
                        
                        forward_return = runner.forward(data, epoch=None, iter_num=None, train=False)
                        if not runner.if_evaluate_on_gpu:
                            forward_return['prediction'] = forward_return['prediction'].detach().cpu()
                            forward_return['target'] = forward_return['target'].detach().cpu()
                            forward_return['inputs'] = forward_return['inputs'].detach().cpu()
                
                        prediction.append(forward_return['prediction'])
                        target.append(forward_return['target'])
                        inputs.append(forward_return['inputs'])
                
                    prediction = torch.cat(prediction, dim=0)
                    target = torch.cat(target, dim=0)
                    inputs = torch.cat(inputs, dim=0)
                
                    returns_all = {'prediction': prediction, 'target': target, 'inputs': inputs}
                    configs[dataset][model][dist][random_state]['returns_all'] = returns_all
    return configs

_configs = get_predictions(_configs)

In [ ]:
# 1. load the model and set the device
_configs = {'ETTh1_PTST_u': {'cfg':'final_weights/PatchTST/univariate/ETTh1_prob.py',
                           'ckpt': 'final_weights/PatchTST/univariate/ETTh1_100_96_720/a8de06edad7530010e0b704422b431a2/PatchTST_best_val_NLL.pt'
                          },
           # 'ETTh1_PTST_q': {'cfg': 'final_weights/PatchTST/quantile/ETTh1_prob.py',
           #                  'ckpt': 'final_weights/PatchTST/quantile/ETTh1_100_96_720/a2a39ac1680165e5ffbda2c7bbda5add/PatchTST_best_val_QL.pt'
           #                 }
          }

random_states = range(5)

configs = {rs:_configs for rs in random_states}

configs = load_runner(configs, random_states=random_states)
configs = get_predictions(configs)
# metrics_results = self.compute_evaluation_metrics(returns_all)

In [ ]:
import torch
def vs_ensemble_torch(obs, fct, p=1.0):
    """
    Compute Variogram Score using PyTorch on GPU.
    obs: shape (..., D)
    fct: shape (..., M, D)
    """
    M = fct.shape[-2]

    # Compute ensemble variogram component
    fct_diff = fct.unsqueeze(-2) - fct.unsqueeze(-1)  # (B, M, D, D)
    # print(fct_diff.shape)
    vfct = (fct_diff.abs() ** p).sum(dim=-3) / M  # (B, D, D)
    # print(vfct.shape)
    # Compute observed variogram component
    obs_diff = obs.unsqueeze(-2) - obs.unsqueeze(-1)  # (B, D, D)
    vobs = (obs_diff.abs() ** p)  # (B, D, D)
    # print(vobs.shape)
    # print(vfct.shape)
    vs = ((vfct - vobs) ** 2).sum(dim=(-2, -1))  # (B,)
    return vs

def es_ensemble_torch(obs: torch.Tensor, fct: torch.Tensor) -> torch.Tensor:
    """
    Compute the energy score using PyTorch.
    
    Parameters:
    - obs: Tensor of shape (B, D)
    - fct: Tensor of shape (B, M, D)

    Returns:
    - Tensor of shape (B,) with energy scores
    """
    M = fct.shape[-2]

    # E_1: mean norm between forecast samples and the observation
    err_norm = torch.norm(fct - obs.unsqueeze(-2), dim=-1)  # (B, M)
    E_1 = err_norm.sum(dim=-1) / M  # (B,)

    # E_2: mean pairwise distance between forecast samples
    spread = fct.unsqueeze(-3) - fct.unsqueeze(-2)  # (B, M, M, D)
    spread_norm = torch.norm(spread, dim=-1)  # (B, M, M)
    E_2 = spread_norm.sum(dim=(-2, -1)) / (M**2) # (B,)

    return E_1 - 0.5 * E_2  # (B,)

def sample(runner, returns_all, random_state=None):
    from prob.prob_head import ProbabilisticHead # load that class for sampling
    head = ProbabilisticHead(1, 1, runner.distribution_type, prob_args=runner.prob_args)
    samples = []
    batch_size = 64
    num_batches = int(returns_all['prediction'].shape[0]/batch_size)+1
    for b in range(num_batches):
        start, end = b*batch_size, min((b+1)*batch_size, returns_all['prediction'].shape[0])
        pred = returns_all['prediction'][start:end, :, :, :]
        sample = head.sample(pred, num_samples=100, random_state=random_state) # [samples x bs x seq_len x nvars]
        sample = sample.permute(1, 0, 2, 3)       # [bs x samples x seq_len x nvars]
        samples.append(sample)
    samples = torch.cat(samples, dim=0)
    return samples

def evaluate(predictions, returns_all, batch_size=4):
    import scoringrules as sr
    import numpy as np
    device = returns_all['target'].device
    targets = returns_all['target'].squeeze(-1)#.detach().cpu()
    sampless = predictions.permute(0, 2, 3, 1)#.detach().cpu() 
    num_batches = int(returns_all['prediction'].shape[0]/batch_size)+1
    # Lists to accumulate metric values
    crps_list = []
    crps_sum_list = []
    vs_05_list = []
    vs_1_list = []
    vs_2_list = []
    es_list = []
    # Loop through batches
    pbar = tqdm(range(num_batches))
    for b in pbar:
        start, end = b * batch_size, min((b + 1) * batch_size, returns_all['prediction'].shape[0])
        if start == end:
            print("SKipping")
            continue  # Skip empty batch
    
        samples = sampless[start:end, :, :, :]
        target = targets[start:end, :, :]
    
        crps = np.mean(sr.crps_ensemble(target.detach().cpu(), samples.detach().cpu(), estimator='pwm'))
        crps_sum = np.mean(sr.crps_ensemble(target.detach().cpu().sum(axis=-1), samples.detach().cpu().sum(axis=-2), estimator='pwm'))
        # vs_05 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=0.5, backend='numba'))
        vs_05 = torch.mean(vs_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device), p=0.5))
        # vs_1 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=1.0, backend='numba'))
        vs_1 = torch.mean(vs_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device), p=1))
        # vs_2 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=2.0, backend='numba'))
        vs_2 = torch.mean(vs_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device), p=2))
        
        # es = np.mean(sr.energy_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), backend='numba'))
        es = torch.mean(es_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device)))
        
        # Append to lists
        crps_list.append(crps)
        crps_sum_list.append(crps_sum)
        vs_05_list.append(vs_05.detach().cpu())
        vs_1_list.append(vs_1.detach().cpu())
        vs_2_list.append(vs_2.detach().cpu())
        es_list.append(es.detach().cpu())
    
    # Final averages
    final_scores = {
        "CRPS": np.mean(crps_list),
        "CRPS_Sum": np.mean(crps_sum_list),
        "VS_0.5": np.mean(vs_05_list),
        "VS_1.0": np.mean(vs_1_list),
        "VS_2.0": np.mean(vs_2_list),
        "ES": np.mean(es_list),
    }
    return final_scores

In [ ]:
def evaluate_all(configs):
    eval_dict = {rs:{} for rs in configs.keys()}
    for rs in configs.keys():
        for key in configs[rs].keys():
            samples = sample(configs[rs][key]['runner'], configs[rs][key]['returns_all'], random_state=rs)
            eval_dict[rs][key] = evaluate(samples, configs[rs][key]['returns_all'], batch_size=4)
    print(eval_dict)

In [ ]:
evaluate_all(configs)

In [ ]:
results = {0: {'ETTh1_PTST_u': {'CRPS': np.float64(2.4197740580153275), 'CRPS_Sum': np.float64(8.704141144967366), 'VS_0.5': np.float32(572256.2), 'VS_1.0': np.float32(14644100.0), 'VS_2.0': np.float32(1985060100000.0), 'ES': np.float32(83.836754)}}, 
           1: {'ETTh1_PTST_u': {'CRPS': np.float64(2.4191808755737285), 'CRPS_Sum': np.float64(8.702462262138733), 'VS_0.5': np.float32(572201.0), 'VS_1.0': np.float32(14609265.0), 'VS_2.0': np.float32(28738159000.0), 'ES': np.float32(83.84088)}}, 
           2: {'ETTh1_PTST_u': {'CRPS': np.float64(2.4195399059040246), 'CRPS_Sum': np.float64(8.704312420450742), 'VS_0.5': np.float32(572196.56), 'VS_1.0': np.float32(14605370.0), 'VS_2.0': np.float32(48451195000.0), 'ES': np.float32(83.866234)}}, 
           3: {'ETTh1_PTST_u': {'CRPS': np.float64(2.41907135491505), 'CRPS_Sum': np.float64(8.702063031509304), 'VS_0.5': np.float32(572468.4), 'VS_1.0': np.float32(14621595.0), 'VS_2.0': np.float32(48899846000.0), 'ES': np.float32(83.85971)}}, 
           4: {'ETTh1_PTST_u': {'CRPS': np.float64(2.4195961071133296), 'CRPS_Sum': np.float64(8.704253091582524), 'VS_0.5': np.float32(572262.1), 'VS_1.0': np.float32(14613185.0), 'VS_2.0': np.float32(138508400000.0), 'ES': np.float32(83.84413)}}}
# Extract metrics
metrics = list(next(iter(results.values()))['ETTh1_PTST_u'].keys())
agg = {metric: [] for metric in metrics}

# rescaling = {
#     "VS_0.5": 1e-4,
#     "VS_1.0": 1e-6,
#     "VS_2.0": 1e-10,
# }
rescaling = {}
for run in results.values():
    for metric in metrics:
        if metric in rescaling.keys():
            agg[metric].append(run['ETTh1_PTST_u'][metric]*rescaling[metric])
        else:
            agg[metric].append(run['ETTh1_PTST_u'][metric])

# Compute stats
summary = {}
for metric in metrics:
    values = np.array(agg[metric], dtype=np.float64)
    summary[metric] = {
        "mean": np.mean(values),
        "std": np.std(values)
    }

# Display results
for metric, stats in summary.items():
    print(f"{metric}: {stats['mean']:.4f} ± {stats['std']:.4f}")

In [ ]:
import scoringrules as sr
import numpy as np
device = returns_all['target'].device
targets = returns_all['target'].squeeze(-1)#.detach().cpu()
sampless = prediction.permute(0, 2, 3, 1)#.detach().cpu() 
print(sampless.shape)
print(targets.shape)
 # 3. Compute approximate metrics
batch_size = 4
num_batches = int(returns_all['prediction'].shape[0]/batch_size)+1
# Lists to accumulate metric values
crps_list = []
crps_sum_list = []
vs_05_list = []
vs_1_list = []
vs_2_list = []
es_list = []

import torch

def vs_ensemble_torch(obs, fct, p=1.0):
    """
    Compute Variogram Score using PyTorch on GPU.
    obs: shape (..., D)
    fct: shape (..., M, D)
    """
    M = fct.shape[-2]

    # Compute ensemble variogram component
    fct_diff = fct.unsqueeze(-2) - fct.unsqueeze(-1)  # (B, M, D, D)
    # print(fct_diff.shape)
    vfct = (fct_diff.abs() ** p).sum(dim=-3) / M  # (B, D, D)
    # print(vfct.shape)
    # Compute observed variogram component
    obs_diff = obs.unsqueeze(-2) - obs.unsqueeze(-1)  # (B, D, D)
    vobs = (obs_diff.abs() ** p)  # (B, D, D)
    # print(vobs.shape)
    # print(vfct.shape)
    vs = ((vfct - vobs) ** 2).sum(dim=(-2, -1))  # (B,)
    return vs

def es_ensemble_torch(obs: torch.Tensor, fct: torch.Tensor) -> torch.Tensor:
    """
    Compute the energy score using PyTorch.
    
    Parameters:
    - obs: Tensor of shape (B, D)
    - fct: Tensor of shape (B, M, D)

    Returns:
    - Tensor of shape (B,) with energy scores
    """
    M = fct.shape[-2]

    # E_1: mean norm between forecast samples and the observation
    err_norm = torch.norm(fct - obs.unsqueeze(-2), dim=-1)  # (B, M)
    E_1 = err_norm.sum(dim=-1) / M  # (B,)

    # E_2: mean pairwise distance between forecast samples
    spread = fct.unsqueeze(-3) - fct.unsqueeze(-2)  # (B, M, M, D)
    spread_norm = torch.norm(spread, dim=-1)  # (B, M, M)
    E_2 = spread_norm.sum(dim=(-2, -1)) / (M**2) # (B,)

    return E_1 - 0.5 * E_2  # (B,)

# Loop through batches
pbar = tqdm(range(num_batches))
for b in pbar:
    start, end = b * batch_size, min((b + 1) * batch_size, returns_all['prediction'].shape[0])
    if start == end:
        print("SKipping")
        continue  # Skip empty batch

    samples = sampless[start:end, :, :, :]
    target = targets[start:end, :, :]

    crps = np.mean(sr.crps_ensemble(target.detach().cpu(), samples.detach().cpu(), estimator='pwm'))
    crps_sum = np.mean(sr.crps_ensemble(target.detach().cpu().sum(axis=-1), samples.detach().cpu().sum(axis=-2), estimator='pwm'))
    # vs_05 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=0.5, backend='numba'))
    vs_05 = torch.mean(vs_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device), p=0.5))
    # vs_1 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=1.0, backend='numba'))
    vs_1 = torch.mean(vs_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device), p=1))
    # vs_2 = np.mean(sr.variogram_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), p=2.0, backend='numba'))
    vs_2 = torch.mean(vs_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device), p=2))
    
    # es = np.mean(sr.energy_score(target.permute(0, 2, 1), samples.permute(0, 2, 3, 1), backend='numba'))
    es = torch.mean(es_ensemble_torch(target.permute(0, 2, 1).to(device), samples.permute(0, 2, 3, 1).to(device)))
    
    # Append to lists
    crps_list.append(crps)
    crps_sum_list.append(crps_sum)
    vs_05_list.append(vs_05.detach().cpu())
    vs_1_list.append(vs_1.detach().cpu())
    vs_2_list.append(vs_2.detach().cpu())
    es_list.append(es.detach().cpu())

    # # Update tqdm with running averages
    # # pbar.set_description(f"CRPS: {np.mean(crps_list):.4f}, VS1: {np.mean(vs_1_list):.4f}, ES: {np.mean(es_list):.4f}")
    
    # pbar.set_description(f"VS: {vs_05:.4f}, VS_T: {vs_05_torch:.4f},")

# Final averages
final_scores = {
    "CRPS": np.mean(crps_list),
    "CRPS_Sum": np.mean(crps_sum_list),
    "VS_0.5": np.mean(vs_05_list),
    "VS_1.0": np.mean(vs_1_list),
    "VS_2.0": np.mean(vs_2_list),
    "ES": np.mean(es_list),
}

print("\nFinal Scores:")
for k, v in final_scores.items():
    print(f"{k}: {v:.4f}")

In [ ]:
Final Scores:
CRPS: 2.4193
CRPS_Sum: 8.7032
VS_0.5: 572217.8750
VS_1.0: 14610408.0000
VS_2.0: 345045532672.0000
ES: 83.8458


Final Scores:
CRPS: 2.4194
CRPS_Sum: 8.7040
VS_0.5: 572181.3125
VS_1.0: 14610432.0000
VS_2.0: 168409481216.0000
ES: 83.8460